In [ ]:
import json
import numpy as np
from utils.utilities import remove_ids, eval_tagging_scores, calculate_accuracy_varying_lengths
import os
import copy

from utils.SemanticMatch import T5ForSoftMatching, BertForSoftMatching, T5V1_1ForSoftMatching
from utils.misc import remove_entity_index,check_alignment,cal_triplet_acc_score_vrdFormer
from utils.eval_func import eval_pred_data
import os

In [ ]:
from data_prep.vidvrd2dataset import VidOR,VidVRD
import os

splits = ["train","test"]
imagenet_vidvrd_root = "D:/Datasets/VidVRD/VidVRD-II/data"
imagenet_vidvrd_video_path = os.path.join(imagenet_vidvrd_root, "videos")

dataset = VidVRD(imagenet_vidvrd_root, imagenet_vidvrd_video_path, splits)
vidvrd_predicates = list(dataset.pred2pid.keys())
vidvrd_objects = list(dataset.so2soid.keys())

vidvrd_predicates = [element.replace("_", " ") for element in vidvrd_predicates]

DATASET_DATA = {
    "dataset_subjects": vidvrd_objects,
    "dataset_objects": vidvrd_objects,
    "dataset_predicates": vidvrd_predicates
}

In [ ]:
with open("inference_outputs_onevision_old\\vidvrd_v14_withid_quad_customprompt\\vidvrd_inference_val_vidvrd_v14_withid_quad_customprompt.json") as f:
    pred_data = eval(json.loads(f.read()))
# with open("inference_outputs_onevision\\[test][onevision]_vidvrd_annotations_v5_3_p01_e01\\vidvrd_inference_val_[test][onevision]_vidvrd_annotations_v5_3_p01_e01.json") as f:
#     pred_data = eval(json.loads(f.read()))

In [ ]:
# with open("inference_outputs_onevision\\vidvrd_v13_indexedlist3\\vidvrd_inference_val_raw_response_vidvrd_v13_indexedlist3.json") as f:
#     raw_data = eval(json.loads(f.read()))
# with open(".\\inference_outputs_onevision\\vidvrd_v14_withid_quad_customprompt\\vidvrd_inference_val_raw_response_vidvrd_v14_withid_quad_customprompt.json") as f:
#     raw_data = eval(json.loads(f.read()))

In [12]:
class PredConfig:
    topk = 1
pred_config = PredConfig()

for R in [1,10,20,50,100]:
    pred_config.topk = R
    before_alignment, all_triplets_pairs, [new_subjects,new_predicates,new_objects] = eval_pred_data(data=pred_data,**DATASET_DATA, PredConfig=pred_config)

    print(f"TOP-{R}",before_alignment["VRDFormer_Logic"]["triplet"])
    

  0%|          | 0/200 [00:00<?, ?it/s]

100%|██████████| 200/200 [00:13<00:00, 14.62it/s]


TOP-1 {'Precision@1': 0.1129, 'Recall@1': 0.0414}


100%|██████████| 200/200 [00:13<00:00, 15.00it/s]


TOP-10 {'Precision@1': 0.0893, 'Recall@1': 0.0963}


...

TOP-20 {'Precision@1': 0.0888, 'Recall@1': 0.0966}


100%|██████████| 200/200 [00:15<00:00, 13.12it/s]


TOP-50 {'Precision@1': 0.0889, 'Recall@1': 0.0968}


100%|██████████| 200/200 [00:14<00:00, 13.64it/s]


TOP-100 {'Precision@1': 0.0889, 'Recall@1': 0.0968}


In [ ]:
new_objects

In [ ]:
def printScore(data):
    score_str = ""
    for key in ["triplet", "subject", "predicate", "object"]:
        precision_before = data["VRDFormer_Logic"][key]["Precision@1"]
        recall_before = data["VRDFormer_Logic"][key]["Recall@1"]
        score_str += f"[{key}: {precision_before:.3f} {recall_before:.3f}]"
    print(score_str)

for model_cls in [T5ForSoftMatching]:
    model = model_cls()
    print("using ",model.__class__)
    model.register_new_gallery(gallery_name="predicates",gallery_data=vidvrd_predicates)
    model.register_new_gallery(gallery_name="objects",gallery_data=vidvrd_objects)

    ALIGNMENT_CONFIDENCE_THR = 0.01 # if less dont replace the entity
    ALIGNMENT_MODEL = model

    new_predicates_aligned = ALIGNMENT_MODEL.align_entities(data_to_align=new_predicates,gallery_name="predicates",confidence_thr=ALIGNMENT_CONFIDENCE_THR)
    new_objects_aligned = ALIGNMENT_MODEL.align_entities(data_to_align=new_objects,gallery_name="objects",confidence_thr=ALIGNMENT_CONFIDENCE_THR)
    new_subjects_aligned = ALIGNMENT_MODEL.align_entities(data_to_align=new_subjects,gallery_name="objects",confidence_thr=ALIGNMENT_CONFIDENCE_THR)

    ALIGNMENT_DATA = {
        'check_alinged': True,
        'alinged_subjects': new_subjects_aligned,
        'alinged_objects': new_objects_aligned,
        'alinged_predicates': new_predicates_aligned
    }



    for R in [1,10,20,50,100]:
        pred_config.topk = R

        after_alignment, all_triplets_pairs_after_alignment, [new_subjects_,new_predicates_,new_objects_] = eval_pred_data(
        pred_data,
        **DATASET_DATA,
        **ALIGNMENT_DATA,PredConfig=pred_config)

        # before_alignment, all_triplets_pairs, [new_subjects,new_predicates,new_objects] = eval_pred_data(data=pred_data,**DATASET_DATA, PredConfig=pred_config)

        print(f"TOP-{R}",after_alignment["VRDFormer_Logic"]["triplet"])

    

    print() 
    printScore(before_alignment)
    print(f"### after alignment with {model.__class__}")
    printScore(after_alignment)
    print(cal_triplet_acc_score_vrdFormer(before_alignment), cal_triplet_acc_score_vrdFormer(after_alignment))
    print()
    del model

C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


using  <class 'utils.SemanticMatch.T5ForSoftMatching'>


 27%|██▋       | 89/335 [00:01<00:05, 48.86it/s]

In [ ]:
before_alignment,after_alignment